In [ ]:
# Data Exploration - Premier League 2024-2025

Notebook này thực hiện khám phá và tiền xử lý dữ liệu Premier League 2024-2025.

## Mục tiêu:
1. Load và kiểm tra dữ liệu
2. Phân tích missing values
3. Thống kê mô tả
4. Feature engineering
5. Chuẩn bị dữ liệu cho các phân tích tiếp theo


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sys
import os

# Thêm thư mục src vào path
sys.path.append('../src')

from data_preprocessing import (
    load_data, explore_data, get_numeric_columns, 
    get_categorical_columns, feature_engineering_players,
    feature_engineering_teams, prepare_data_for_analysis
)

warnings.filterwarnings('ignore')

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Đã import các thư viện và modules cần thiết")


## 1. Load dữ liệu


In [ ]:
# Load tất cả dữ liệu
data = load_data()

print("📊 Dữ liệu đã được load:")
for name, df in data.items():
    print(f"  - {name}: {df.shape[0]} rows, {df.shape[1]} columns")

# Lưu vào biến riêng để dễ sử dụng
players_df = data['players']
keepers_df = data['keepers']
teams_for_df = data['teams_for']
teams_vs_df = data['teams_vs']


## 2. Khám phá dữ liệu cầu thủ (Players)


In [ ]:
# Khám phá dữ liệu cầu thủ
missing_players = explore_data(players_df, "Players Dataset")

print("\n" + "="*60)
print("SAMPLE DATA - First 3 rows:")
print("="*60)
print(players_df.head(3))

print("\n" + "="*60)
print("COLUMN NAMES:")
print("="*60)
print(f"Total columns: {len(players_df.columns)}")
print("\nFirst 20 columns:")
print(players_df.columns[:20].tolist())


In [ ]:
# Thống kê mô tả cho các cột số quan trọng
numeric_cols = get_numeric_columns(players_df, exclude_cols=['Age', 'Born'])

# Chọn một số cột quan trọng để xem
important_cols = [col for col in numeric_cols if any(keyword in col.lower() 
    for keyword in ['gls', 'ast', 'xg', 'xa', 'sh', 'sot', 'pass', 'tkl', 'touches'])]

if len(important_cols) > 0:
    print("📈 Thống kê mô tả - Các chỉ số quan trọng:")
    print(players_df[important_cols[:15]].describe())


In [ ]:
# Phân tích vị trí cầu thủ
if 'Pos' in players_df.columns:
    print("📊 Phân bố vị trí cầu thủ:")
    print(players_df['Pos'].value_counts())
    
    plt.figure(figsize=(10, 6))
    players_df['Pos'].value_counts().plot(kind='bar', color='steelblue')
    plt.title('Phân bố vị trí cầu thủ', fontsize=14, fontweight='bold')
    plt.xlabel('Vị trí', fontweight='bold')
    plt.ylabel('Số lượng', fontweight='bold')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('../results/position_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()


## 3. Khám phá dữ liệu đội bóng (Teams)


In [ ]:
# Khám phá dữ liệu đội bóng
explore_data(teams_for_df, "Teams For Dataset")
explore_data(teams_vs_df, "Teams VS Dataset")

print("\n" + "="*60)
print("TEAMS FOR - Sample:")
print("="*60)
print(teams_for_df.head())

print("\n" + "="*60)
print("TEAMS VS - Sample:")
print("="*60)
print(teams_vs_df.head())


## 4. Feature Engineering


In [ ]:
# Tạo các features mới cho cầu thủ
players_df_enhanced = feature_engineering_players(players_df)

print("✅ Đã tạo các features mới cho cầu thủ")
print(f"\nSố cột trước: {len(players_df.columns)}")
print(f"Số cột sau: {len(players_df_enhanced.columns)}")
print(f"\nCác cột mới được thêm:")
new_cols = set(players_df_enhanced.columns) - set(players_df.columns)
print(list(new_cols))


In [ ]:
# Tạo các features mới cho đội bóng
teams_merged = feature_engineering_teams(teams_for_df, teams_vs_df)

if teams_merged is not None:
    print("✅ Đã merge và tạo features mới cho đội bóng")
    print(f"\nShape: {teams_merged.shape}")
    print("\nSample data:")
    print(teams_merged.head())


## 5. Lưu dữ liệu đã xử lý


In [ ]:
# Chuẩn bị dữ liệu cho phân tích
players_processed = prepare_data_for_analysis(players_df_enhanced)

# Lưu dữ liệu đã xử lý
players_processed.to_excel('../data/players_processed.xlsx', index=False)
print("✅ Đã lưu players_processed.xlsx")

if teams_merged is not None:
    teams_processed = prepare_data_for_analysis(teams_merged, target_cols=['Squad'])
    teams_processed.to_excel('../data/teams_processed.xlsx', index=False)
    print("✅ Đã lưu teams_processed.xlsx")

print("\n🎉 Hoàn tất khám phá dữ liệu!")
